In [1]:
import numpy as np
import pandas as pd
import itertools

from scipy import stats
from statsmodels.stats.descriptivestats import sign_test
from statsmodels.stats.weightstats import zconfint
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [7]:
patients_data=np.array([49,58,75,110,112,132,151,276,281,362])

In [8]:
stats.wilcoxon(patients_data-200)

WilcoxonResult(statistic=17.0, pvalue=0.28450269791120752)

In [9]:
forest_non=np.array([22,22,15,13,19,19,18,20,21,13,13,15])
forest_yes=np.array([17,18,18,15,12,4,14,15,10])

In [10]:
stats.mannwhitneyu(forest_yes,forest_non,alternative='less')

MannwhitneyuResult(statistic=27.0, pvalue=0.029004992720873729)

In [31]:
data=pd.read_csv("Challenger.txt",delimiter="\t")

In [32]:
data

,Unnamed: 0,Temperature,Incident
0,Apr12.81,18.9,0
1,Nov12.81,21.1,1
2,Mar22.82,20.6,0
3,Nov11.82,20.0,0
4,Apr04.83,19.4,0
5,Jun18.83,22.2,0
6,Aug30.83,22.8,0
7,Nov28.83,21.1,0
8,Feb03.84,13.9,1
9,Apr06.84,17.2,1


In [33]:
data.describe()

,Temperature,Incident
count,23.000000,23.000000
mean,20.860870,0.304348
std,3.919501,0.470472
min,11.700000,0.000000
25%,19.400000,0.000000
50%,21.100000,0.000000
75%,23.900000,1.000000
max,27.200000,1.000000


In [39]:
temperature_destroy=np.array(data[data.Incident==1]["Temperature"])
temperature=np.array(data[data.Incident==0]["Temperature"])

In [43]:
def get_bootstrap_samples(data, n_samples):
    indices = np.random.randint(0, len(data), (n_samples, len(data)))
    samples = data[indices]
    return samples

In [44]:
def stat_intervals(stat, alpha):
    boundaries = np.percentile(stat, [100 * alpha / 2., 100 * (1 - alpha / 2.)])
    return boundaries

In [53]:
np.random.seed(0)
temperature_destroy_mean_scores=np.array(map(np.mean,get_bootstrap_samples(temperature_destroy,1000)))
temperature_mean_scores=np.array(map(np.mean,get_bootstrap_samples(temperature,1000)))

In [54]:
stat_intervals(temperature_destroy_mean_scores-temperature_mean_scores,0.05)

array([-8.06457589, -1.45040179])

In [55]:
def permutation_t_stat_ind(sample1, sample2):
    return np.mean(sample1) - np.mean(sample2)

In [56]:
def get_random_combinations(n1, n2, max_combinations):
    index = range(n1 + n2)
    indices = set([tuple(index)])
    for i in range(max_combinations - 1):
        np.random.shuffle(index)
        indices.add(tuple(index))
    return [(index[:n1], index[n1:]) for index in indices]

In [57]:
def permutation_zero_dist_ind(sample1, sample2, max_combinations = None):
    joined_sample = np.hstack((sample1, sample2))
    n1 = len(sample1)
    n = len(joined_sample)
    
    if max_combinations:
        indices = get_random_combinations(n1, len(sample2), max_combinations)
    else:
        indices = [(list(index), filter(lambda i: i not in index, range(n))) \
                    for index in itertools.combinations(range(n), n1)]
    
    distr = [joined_sample[list(i[0])].mean() - joined_sample[list(i[1])].mean() \
             for i in indices]
    return distr

In [58]:
def permutation_test(sample, mean, max_permutations = None, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    t_stat = permutation_t_stat_ind(sample, mean)
    
    zero_distr = permutation_zero_dist_ind(sample, mean, max_permutations)
    
    if alternative == 'two-sided':
        return sum([1. if abs(x) >= abs(t_stat) else 0. for x in zero_distr]) / len(zero_distr)
    
    if alternative == 'less':
        return sum([1. if x <= t_stat else 0. for x in zero_distr]) / len(zero_distr)

    if alternative == 'greater':
        return sum([1. if x >= t_stat else 0. for x in zero_distr]) / len(zero_distr)

In [59]:
np.random.seed(0)
permutation_test(temperature_destroy,temperature,max_permutations=10000)

0.0057000000000000002